In [ ]:
import numpy as np
import json
from sklearn.metrics import roc_auc_score
import shutil
import warnings
import cv2
import pandas as pd
from PIL import Image
from skimage.transform import resize
from imgaug import augmenters as iaa
import os
import pickle
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

import keras.backend as kb
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.utils import Sequence
from keras.layers import Input
from keras.layers import BatchNormalization, Input, Dropout
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet121
warnings.filterwarnings("ignore")

In [ ]:
#thread safe generators with augmentation
class AugmentedImageSequence(Sequence):
    def __init__(self, dataset_file, class_names, source_dir, batch_size = 8, target_size = (224,224),
                augmenter = None, verbose = 0, steps = None, shuffle_on_epoch_end = True, random_state = 3):
        self.dataset_df = pd.read_csv(dataset_file)
        self.source_dir = source_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.augmenter = augmenter
        self.verbose = verbose
        self.shuffle = shuffle_on_epoch_end
        self.random_state = random_state
        self.class_names = class_names
        self.prepare_dataset()
        if steps is None:
            self.steps = int(np.ceil(len(self.x_path) / float(self.batch_size)))
        else:
            self.steps = int(steps)
        
    def __bool__(self):
        return True

    def __len__(self):
        return self.steps

    def __getitem__(self, idx):
        batch_x_path = self.x_path[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = np.asarray([self.load_image(x_path) for x_path in batch_x_path])
        batch_x = self.transform_batch_images(batch_x)
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
    def load_image(self, image_file):
        image_path = os.path.join(self.source_dir, image_file)
        image = Image.open(image_path)
        image_array = np.asarray(image.convert("RGB"))
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img_temp = np.zeros((image_array.shape[0],image_array.shape[1],image_array.shape[2]))

        for i in range (3):
            img_temp[:,:,i] = clahe.apply(image_array[:,:,i])

        image_array = img_temp / 255.
        image_array = resize(image_array, self.target_size)
        return image_array

    def transform_batch_images(self, batch_x):
        if self.augmenter is not None:
            batch_x = self.augmenter.augment_images(batch_x)
        imagenet_mean = np.array([0.485, 0.456, 0.406])
        imagenet_std = np.array([0.229, 0.224, 0.225])
        batch_x = (batch_x - imagenet_mean) / imagenet_std
        return batch_x

    def get_y_true(self):
        if self.shuffle:
            raise ValueError("""
            You're trying run get_y_true() when generator option 'shuffle_on_epoch_end' is True.
            """)
        return self.y[:self.steps*self.batch_size, :]

    def prepare_dataset(self):
        df = self.dataset_df.sample(frac=1., random_state=self.random_state)
        self.x_path, self.y = df["Image_Location"].values, df[self.class_names].values

    def on_epoch_end(self):
        if self.shuffle:
            self.random_state += 1
            self.prepare_dataset()


In [ ]:
def get_sample_counts(output_dir, dataset, class_names):
    df = pd.read_csv(os.path.join(output_dir, f"{dataset}.csv"))
    total_count = df.shape[0]
    labels = df[class_names].values
    positive_counts = np.sum(labels, axis=0)
    class_positive_counts = dict(zip(class_names, positive_counts))
    return total_count, class_positive_counts

In [ ]:
#this section calculates the AUROC values for the validation/dev set
class MultipleClassAUROC(Callback):
    """
    Monitor mean AUROC and update model
    """
    def __init__(self, sequence, class_names, output_path, stats=None, workers=1):
        super(Callback, self).__init__()
        self.sequence = sequence
        self.workers = workers
        self.class_names = class_names
        self.output_path = output_path
        
        self.best_auroc_log_path = os.path.join(output_path, "best_auroc.log")
        self.stats_output_path = os.path.join(output_path, "training_stats.json")
        # for resuming previous training
        if stats:
            self.stats = stats
        else:
            self.stats = {"best_mean_auroc": 0}

        # aurocs log
        self.aurocs = {}
        for c in self.class_names:
            self.aurocs[c] = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Calculate the average AUROC and save the best model weights according
        to this metric.

        """
        print("\n*********************************")
        self.stats["lr"] = float(kb.eval(self.model.optimizer.lr))
        print(f"current learning rate: {self.stats['lr']}")

        """
        y_hat shape: (#samples, len(class_names))
        y: [(#samples, 1), (#samples, 1) ... (#samples, 1)]
        """
        y_hat = self.model.predict_generator(self.sequence, workers=self.workers)
        y = self.sequence.get_y_true()

        print(f"*** epoch#{epoch + 1} dev auroc ***")
        current_auroc = []
        for i in range(len(self.class_names)):
            try:
                score = roc_auc_score(y[:, i], y_hat[:, i])
            except ValueError:
                score = 0
            self.aurocs[self.class_names[i]].append(score)
            current_auroc.append(score)
            print(f"{i+1}. {self.class_names[i]}: {score}")
        print("*********************************")

        # customize your multiple class metrics here
        mean_auroc = np.mean(current_auroc)
        print(f"mean auroc: {mean_auroc}")
        return

In [ ]:
augmenter = iaa.Sequential([iaa.OneOf([
        iaa.Fliplr(0.5),
        iaa.Affine(rotate=(-10, 10)),
        iaa.Noop(),
        iaa.GaussianBlur(sigma=(0.0, 1.0)),
        iaa.Affine(shear=(-16, 16)),
        iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)})
    ])],random_order=True)

In [ ]:
image_size = 512
batch_size = 4
class_names = "Normal,Pneumonia"
class_names = class_names.split(",")
output_dir = "../output/"
image_dir = "../data/images/"
epochs = 20
init_lr = 1e-4
min_lr = 1e-8
csv_dir = "../data/"
generator_workers = 8
use_trained_model_weights = False

In [ ]:
if use_trained_model_weights:
    # resuming mode
    print("** use trained model weights **")
    # load training status for resuming
    training_stats_file = os.path.join(output_dir, "training_stats.json")
    if os.path.isfile(training_stats_file):
        # TODO: add loading previous learning rate?
        training_stats = json.load(open(training_stats_file))
    else:
        training_stats = {}
else:
    # start over
    training_stats = {}

In [ ]:
train_counts, train_pos_counts = get_sample_counts(csv_dir, "train", class_names)
val_counts, val_pos_counts = get_sample_counts(csv_dir, "val", class_names)
test_counts, test_pos_counts = get_sample_counts(csv_dir, "test_onehot", class_names)

In [ ]:
print(train_counts)
print(train_pos_counts)
print(val_counts)
print(val_pos_counts)
print(test_counts)
print(test_pos_counts)

In [ ]:
train_steps = int(train_counts / batch_size)
validation_steps = int(val_counts / batch_size)

In [ ]:
#model definition
def chexnet_model(input_shape, num_classes):
    input_tensor = Input(shape = input_shape, name = 'input_1')
    
    base_model = DenseNet121(include_top = False, weights = 'imagenet', input_shape = input_shape, pooling = "avg")
    X = base_model(input_tensor)
    X = Dropout(0.2)(X)
    output = Dense(num_classes, activation = 'sigmoid', name = 'dense_1', kernel_initializer = 'he_normal')(X)
    model = Model(input_tensor, output)
    
    return model

In [ ]:
#we train on the images in this section
train_sequence = AugmentedImageSequence(
            dataset_file=os.path.join(csv_dir, "train.csv"),
            class_names=class_names,
            source_dir=image_dir,
            batch_size=batch_size,
            target_size=(image_size, image_size),
            augmenter=augmenter,
            steps=train_steps)

validation_sequence = AugmentedImageSequence(
            dataset_file=os.path.join(csv_dir, "val.csv"),
            class_names=class_names,
            source_dir=image_dir,
            batch_size=batch_size,
            target_size=(image_size, image_size),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False)

checkpoint = ModelCheckpoint('../models/chexnet_v4_rsna_{epoch:02d}.h5', verbose = 1, save_weights_only = True, period = 1)

auroc = MultipleClassAUROC(
            sequence=validation_sequence,
            class_names=class_names,
            output_path=output_dir,
            stats=training_stats,
            workers=generator_workers)
callbacks = [checkpoint,TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3,verbose=1, mode="min", min_lr=min_lr), auroc]


model = chexnet_model(input_shape = (image_size,image_size,3), num_classes = len(class_names))

#model.summary() 7 mil params

#change the values below based on what you want to do. warmup is only required the first time you start training.
#load saved weights if you want to continue training from a previously stopped point.
train_mode = True
warmup_model = True
load_weights = False

if train_mode:
    
    #load previously saved weights if you want to continue training
    if load_weights:
        model.load_weights('../models/chexnet_v3_rsna_04.h5', by_name = True)
    
    if warmup_model:
        for layer in model.layers:
            layer.trainable = False

        model.layers[-1].trainable = True
        
        model.compile(loss = 'binary_crossentropy', optimizer=Adam(lr = init_lr))

        model.fit_generator(generator = train_sequence, steps_per_epoch = train_steps, epochs=1, verbose=1, 
                            workers=generator_workers,shuffle = False)
    
     #now train all the layers
    for layer in model.layers:
        layer.trainable = True
        
    model.compile(loss = 'binary_crossentropy', optimizer=Adam(lr = init_lr))

    history = model.fit_generator(generator = train_sequence, steps_per_epoch = train_steps, epochs=epochs, verbose=1, 
                        workers=generator_workers,shuffle = False, validation_data=validation_sequence,
                        validation_steps=validation_steps, callbacks=callbacks)

with open(os.path.join(output_dir, "history.pkl"), "wb") as f:
            pickle.dump({
                "history": history.history,
                "auroc": auroc.aurocs,
            }, f)

In [ ]:
# code from here on generates the cam output and auroc scores
output_dir_test = "../output/cam/"
image_source_dir = "../data/images/"
image_dimension = 512


# CAM config
bbox_list_file = "../data/test.csv"

print("** load model **")
model = chexnet_model(input_shape = (image_dimension,image_dimension,3), num_classes = len(class_names))
model.load_weights('../models/chexnet_v4_rsna_03.h5', by_name = True)

In [ ]:
#the code in this section generates the cam images.
df_images = pd.read_csv(bbox_list_file) 

#for the image index below change to a specific number manually or generate images based on random index values
img_index = 226 #np.random.randint(0, 1450)
print(img_index)
file_name = df_images["Image_Location"][img_index]
print(file_name)

# draw bbox with labels
# draw bbox with labels
img_ori = Image.open(os.path.join(image_source_dir, file_name))
img_ori = np.asarray(img_ori.convert("RGB"))

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img_temp = np.zeros((img_ori.shape[0],img_ori.shape[1],img_ori.shape[2]))

for i in range (3):
    img_temp[:,:,i] = clahe.apply(img_ori[:,:,i])

img_ori = img_temp / 255.
img_ori = resize(img_ori, (512,512))

img_transformed = np.array([img_ori])

label = df_images["Label"][img_index]

index = class_names.index(label)

# CAM overlay
class_weights = model.layers[-1].get_weights()[0]

layer_model_1 = Model(inputs=model.layers[1].layers[0].input, outputs = model.layers[1].layers[-2].output)

print(img_transformed.shape)
conv_outputs = layer_model_1.predict(img_transformed)[0]
predictions = model.predict(img_transformed)[0]
print(predictions)
pred_name = class_names[np.argmax(predictions)]
print(pred_name)
output_path = os.path.join(output_dir_test, f"{label}_{pred_name}_{img_index}_{file_name}")

# Create the class activation map.
cam = np.zeros(dtype=np.float32, shape=(conv_outputs.shape[:2]))
for i, w in enumerate(class_weights[index]):
    cam += w * conv_outputs[:, :, i]

cam /= np.max(cam)
cam = cv2.resize(cam, img_ori.shape[:2])
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
heatmap[np.where(cam < 0.2)] = 0
img = heatmap*0.5 + img_ori

# add label & rectangle
ratio = 1

if not np.isnan(df_images["x1"][img_index]):
    x1 = int(df_images["x1"][img_index] * ratio)
    y1 = int(df_images["y1"][img_index] * ratio)
    x2 = int((df_images["x2"][img_index] ) * ratio)
    y2 = int((df_images["y2"][img_index] ) * ratio)
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.putText(img, text=label, org=(5, 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.8, color=(0, 0, 255), thickness=1)
cv2.imwrite(output_path, img)

In [ ]:
#use this to calculate ytrue and ypred on test set

df_images = pd.read_csv(bbox_list_file)

y_true = []
y_pred = []
y_pred_raw = []

for img_index in range(df_images.shape[0]): 

    file_name = df_images["Image_Location"][img_index]

    # draw bbox with labels
    img_ori = Image.open(os.path.join(image_source_dir, file_name))
    img_ori = np.asarray(img_ori.convert("RGB"))
           
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_temp = np.zeros((img_ori.shape[0],img_ori.shape[1],img_ori.shape[2]))

    for i in range (3):
        img_temp[:,:,i] = clahe.apply(img_ori[:,:,i])


    img_ori = img_temp / 255.
    img_ori = resize(img_ori, (512,512))
    #imshow(img_ori)

    label = df_images["Label"][img_index]

    index = class_names.index(label)
    
    if index == 0:
        index = [1, 0]
    elif index == 1:
        index = [0, 1]
        
    y_true.append(index)
    
   # img_transformed = resize(img_ori, (image_dimension,image_dimension,3))
    img_transformed = np.array([img_ori])

    predictions = model.predict(img_transformed)[0]
    y_pred_raw.append(predictions)
    
    if np.argmax(predictions) == 0:
        y_pred.append([1, 0])
    else:
        y_pred.append([0, 1])

In [ ]:
#get the auroc score in this section from ytrue and ypred

y_true = np.array(y_true)
y_pred = np.array(y_pred)

for i in range(len(class_names)):
    try:
        score = roc_auc_score(y_true[:, i], y_pred[:, i])
    except ValueError:
        score = 0
    print(f"{i+1}. {class_names[i]}: {score}")